In [ ]:
import tensorflow as tf

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import tensorflow as tf
from IPython import display
from utils import ideal_crop 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from ast import literal_eval
from tqdm import tqdm_notebook
import pathlib
import helper
import tensorflow.keras as keras
from sklearn.utils import shuffle
from glob import glob
import cv2
from tensorflow.keras.models import model_from_json
import json
from tensorflow.keras.models import Model
# from layer import *

In [3]:
def crop_resize_image(image):
    std_height, std_width = 544, 960
    image = np.array(image).astype('float32')/255.
    height, width = image.shape[:2]
    if (height/std_height)<=(width/std_width):
        r = height/std_height
    else:
        r = width/std_width
    new_height, new_width = int(std_height*r), int(std_width*r)
    h_center, w_center = height//2, width//2
        
    image = image[int(h_center-(new_height//2)):int(h_center-(new_height//2)+new_height), 
                  int(w_center-(new_width//2)):int(w_center-(new_width//2)+new_width)]
    return image



In [4]:
imgs_list = glob('/home/jovyan/ta-hsi-datacenter/resize_544_9600/mountain/*.jpg')
imgs = []
for i in tqdm_notebook(imgs_list):
    img = cv2.imread(i)[:,:,::-1]
    img = crop_resize_image(img)
    img = np.array(img, dtype=np.float16)
    imgs.append(img)
imgs = np.array(imgs, dtype=np.float16)

imgs_arr = [imgs]

In [50]:
# imgs[0].shape

In [8]:

with open('../../ta-hsi-datacenter/output_results/model/resize_68_120/G1_68_120.json', 'r') as json_file:
    temp = json_file.read()
    G1 = model_from_json(temp)
    G1.load_weights('../../ta-hsi-datacenter/output_results/weight/resize_68_120/g1.h5')



    
    
with open('../../ta-hsi-datacenter/output_results/model/resize_68_120/D1_68_120.json', 'r') as json_file:
    temp = json_file.read()
    old_D1 = model_from_json(temp)
    old_D1.load_weights('../../ta-hsi-datacenter/output_results/weight/resize_68_120/d1.h5')
    
    
    
# with open('../../ta-hsi-datacenter/output_results/model/resize_68_120/GAN1_68_120.json', 'r') as json_file:
#     temp = json_file.read()
#     GAN1 = model_from_json(temp)
#     GAN1.load_weights('../../ta-hsi-datacenter/output_results/weight/resize_68_120/gan1.h5')
    




In [6]:
# GAN1.summary()

In [9]:
def residual_block(res_input,conv_dim):
    x = layers.Conv2D(conv_dim, 3, strides=1, padding='same')(res_input)
    x = layers.ReLU()(x)
    x = layers.Conv2D(conv_dim, 3, strides=1, padding='same')(x)
    x = layers.add([x, res_input])
    x = layers.ReLU()(x)

    return x

In [10]:
def add_g1_blocklayers(model):
    out = model.layers[-3]

    x = residual_block(out.output,128)
    x = layers.UpSampling2D(size=(2, 2))(out.output)
    x = layers.Conv2DTranspose(64,(3,3),strides=(1,1),padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)   
    x = layers.Conv2DTranspose(64,(3,3),strides=(1,1),padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    result = layers.Conv2DTranspose(3,(1,1),strides=(1,1),padding='same',activation='linear')(x)

    model = Model(model.input,result)
    return model

In [136]:
# def add_d1_blocklayers(model):
    
#     discriminator_input = layers.Input(shape=(136, 240, 3))

#     x = layers.Conv2D(64, 1, padding = 'same')(discriminator_input)
#     x = layers.LeakyReLU(0.2)(x)
#     x = layers.Conv2D(64, 3, padding = 'same')(x)
#     x = layers.LeakyReLU(0.2)(x)
#     x = layers.Conv2D(128, 3, padding = 'same')(x)
#     x = layers.LeakyReLU(0.2)(x)
#     x = layers.AveragePooling2D()(x)
#     x = residual_block(x,128)
    

#     model = Model(temp_model.input,x)
#     return model

In [11]:
G1.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 130560)            26242560  
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 130560)            0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 17, 30, 256)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 34, 60, 256)       0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 34, 60, 256)       590080    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 34, 60, 256)       0   

In [8]:
G1.trainable=False

In [12]:
G1 = add_g1_blocklayers(G1)

In [34]:
# D1.summary()

In [13]:
G1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 130560)            26242560  
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 130560)            0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 17, 30, 256)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 34, 60, 256)       0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 34, 60, 256)       590080    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 34, 60, 256)       0     

In [81]:
# D1.summary()

In [14]:
import tensorflow.keras.layers as layers

def build_generator(latent_dim, output_size):
    filter_num = [256, 128, 64 , 32]
    generator_input = keras.Input(shape=(latent_dim,))
    
    
    height, width = output_size
    
    x = layers.Dense(filter_num[0] * int(height//4) * int(width//4))(generator_input)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Reshape((int(height//4), int(width//4), filter_num[0]))(x) 
    
    #### 17*30*256
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2DTranspose(filter_num[0],(3,3),strides=(1,1),padding='same')(x)  
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2DTranspose(filter_num[0],(3,3),strides=(1,1),padding='same' )(x)
    x = layers.LeakyReLU(0.2)(x)

    #### 34*60*128
           
#     x = residual_block(x,filter_num[0])
#     x = residual_block(x,filter_num[0])
#     x = layers.Conv2DTranspose(filter_num[1],(5,5),strides=(2,2),padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2DTranspose(filter_num[1],(3,3),strides=(1,1),padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)   
    x = layers.Conv2DTranspose(filter_num[1],(3,3),strides=(1,1),padding='same')(x)
    x = layers.LeakyReLU(0.2)(x) 

    
    
#     x = layers.UpSampling2D(size=(2, 2))(x)
#     x = layers.Conv2DTranspose(filter_num[2],(3,3),strides=(1,1),padding='same')(x)
#     x = layers.LeakyReLU(0.2)(x)   
#     x = layers.Conv2DTranspose(filter_num[2],(3,3),strides=(1,1),padding='same')(x)
#     x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2DTranspose(3,(1,1),strides=(1,1),padding='same',activation='linear')(x)
    
    
    generator = keras.models.Model(generator_input,x)
    
    return keras.models.Model(generator_input,x)

In [23]:
G1 = build_generator(200, (544,960))
G1.summary()

NameError: name 'build_generator' is not defined

In [18]:
def build_discriminator(input_size):
    height, width, channels = input_size
    filter_num = [128,256]
    
    discriminator_input = layers.Input(shape=(height, width, channels))
    x = layers.Conv2D(64, 1, padding = 'same')(discriminator_input)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(64, 3, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(128, 3, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.AveragePooling2D()(x)
#     x = residual_block(x,128)


    
    
    x = layers.Conv2D(filter_num[0], 3, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(filter_num[0], 3, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.AveragePooling2D()(x)
    
    #### 272*480*16
#     x = MinibatchStatConcatLayer(averaging='all', name='Dstat')(x)
#     x = add_minibatch_stddev_feat(x)
    x = layers.Conv2D(filter_num[1], 3, strides = 1, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(filter_num[1], 4, strides = 1, padding = 'same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.AveragePooling2D()(x)
    


    x = layers.Flatten()(x)

    x = layers.Dropout(0.4)(x)


    x = layers.Dense(1, activation='sigmoid')(x)

    discriminator = keras.models.Model(discriminator_input, x)
    return discriminator

In [11]:
# D_split = [3,5,7,9,14]

In [19]:
old_D1.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 68, 120, 3)]      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 68, 120, 128)      3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 68, 120, 128)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 68, 120, 128)      147584    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 68, 120, 128)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 68, 120, 128)      147584    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 68, 120, 128)      0   

In [20]:
D1 = build_discriminator((136, 240,3))
D1.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 136, 240, 3)]     0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 136, 240, 64)      256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 136, 240, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 136, 240, 64)      36928     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 136, 240, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 136, 240, 128)     73856     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 136, 240, 128)     0   

In [21]:
temp_weights = [layer.get_weights() for layer in old_D1.layers[3:]]

In [24]:
## residual block
# for i in range(len(temp_weights)):
#     D1.layers[i+13].set_weights(temp_weights[i])
    

for i in range(len(temp_weights)):
    D1.layers[i+8].set_weights(temp_weights[i])

In [95]:
len(G1.trainable_weights)

8

In [16]:
for l in D1.layers[13:]:
    l.trainable = False

In [94]:
for l in G1.layers[-7:]:
    l.trainable = False

In [25]:
def build_GAN(G, D):
    D.trainable = False
    gan_input = G.input
    gan_output = D(G(gan_input))
    gan = keras.models.Model(gan_input, gan_output)
    return gan

In [26]:
GAN = build_GAN(G1, D1)
GAN.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
model (Model)                (None, 136, 240, 3)       27976259  
_________________________________________________________________
model_2 (Model)              (None, 1)                 1880769   
Total params: 29,857,028
Trainable params: 27,976,259
Non-trainable params: 1,880,769
_________________________________________________________________


In [27]:
iterations = 10000
latent_dim = 200
height_1, width_1 = 136, 240
height_2, width_2 = 544, 960

# G1 = build_generator(latent_dim, (height_1, width_1))
# D1 = build_discriminator((height_1, width_1, 3))
# optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
D1.compile(loss = 'binary_crossentropy', optimizer = optimizer)

GAN1 = build_GAN(G1, D1)
optimizer = keras.optimizers.Adam(lr = 0.0001, beta_1 = 0.5)
GAN1.compile(loss = 'binary_crossentropy', optimizer = optimizer)

In [45]:
model_json = G1.to_json()
os.path
with open("generator.json","w") as json_file:
    json_file.write(model_json)
    


In [115]:
# import os
# from keras.preprocessing import image
# import time

# start_time_all = time.time()


# save_dir = '../../ta-hsi-datacenter/output_results/result_image'
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# # Start training loop
# start = 0
# start_time = time.time()

# low_iteration = 600
# high_iteration = 1000
# pre_low_step = 55300
# pre_high_step = 5700
# batch_size = 16
# for step in range(iterations):
#     for low_step in range(low_iteration):
#         real_images = next(datagen_train_2x)[0]
# #         stop = start+batch_size
# #         real_images = x_train[start: stop]
# #         batch_size = real_images.shape[0]
#         random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        
        
#         generated_images = G1.predict(random_latent_vectors)
        
#         labels = np.concatenate([np.zeros((batch_size, 1)),
#                              np.ones((batch_size, 1))])

#         labels_real = np.ones((batch_size, 1)) - 0.05*np.abs(np.random.random((batch_size, 1)))
#         labels_fake = np.zeros((batch_size, 1)) + 0.05*np.abs(np.random.random((batch_size, 1)))

#         d_loss_real = D1.train_on_batch(real_images, labels_real)
#         d_loss_fake = D1.train_on_batch(generated_images, labels_fake)
#         d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

#         random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

#         misleading_targets = np.ones((batch_size, 1))

#         g_loss = GAN1.train_on_batch(random_latent_vectors, misleading_targets)
# #         start += batch_size
# #         if start > len(x_train) - batch_size:
# #             start = 0

#         if low_step % 100 == 0:
#             # Save model weights
#             GAN1.save_weights('../../ta-hsi-datacenter/output_results/weight/gan1.h5')
#             G1.save_weights('../../ta-hsi-datacenter/output_results/weight/g1.h5')
#             D1.save_weights('../../ta-hsi-datacenter/output_results/weight/d1.h5')

#             # Print metrics
#             print('low resolution, discriminator loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, d_loss))
#             print('low resolution, adversarial loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, g_loss))
#             display_grid = np.zeros((4*height_1,width_1,3))
        
#             for j in range(4):
#                 display_grid[j*height_1:(j+1)*height_1,0:width_1,:] = generated_images[j]
        
#             img = image.array_to_img((display_grid[:,:,::-1]*127.5)+127.5, scale=False)
#             img.save(os.path.join(save_dir, 'low_generated_' + str(step*low_iteration+low_step+pre_low_step) + '.png'))
#             print("--- %s seconds ---" % (time.time() - start_time))
#             start_time = time.time()

In [116]:
#G_split = [7,9,11,13,15]
#D_split = [3,5,7,9,14]

In [117]:
split_layers = {0:[7,15,3,14],1:[9,15,5,14],2:[11,15,7,14],3:[13,15,9,14],4:[15,15,14,14]}

In [118]:
# plt.imshow(np.array(imgs_arr[0][0]*255,dtype=np.int))

In [28]:
import os
from keras.preprocessing import image
import time

start_time_all = time.time()


save_dir = '../../ta-hsi-datacenter/output_results/result_image/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Start training loop
start = 0
start_time = time.time()

low_iteration = 600
high_iteration = 1000
pre_low_step = 0
pre_high_step = 0
batch_size = 8

batch_num = len(imgs_arr[0])//batch_size
# imgs_temp = imgs[:batch_size*batch_num]


for idx,imgs in enumerate(imgs_arr):
    imgs_temp = imgs[:batch_size*batch_num]
    
    for step in range(iterations):
        imgs_temp = shuffle(imgs_temp)

    #     ## 每一次epoch 都要慢慢解鎖layer
    #     for num_split,split_loc_arr in split_layers.items():
    #         imgs_temp = imgs_arr[num_split][:batch_size*batch_num]
    #         imgs_temp = shuffle(imgs_temp)
    #         if num_split!=4:
    #             ###############
    #             g_split_start , g_split_end , d_split_start , d_split_end = split_loc_arr[0],split_loc_arr[1],split_loc_arr[2],split_loc_arr[3]
    #             ## generator部分上鎖
    #             for k in range(g_split_start,g_split_end):
    #                 G1.layers[k].trainable=False
    #             ## discriminator部分上鎖
    #             for k in range(d_split_start,d_split_end):
    #                 D1.layers[k].trainable=False
    #             ###############    

        for low_step in range(batch_num):
            real_images = imgs_temp[low_step*batch_size:(low_step+1)*batch_size]
            real_images = (real_images-0.5)*2
            random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))


            generated_images = G1.predict(random_latent_vectors)

            labels = np.concatenate([np.zeros((batch_size, 1)),
                                 np.ones((batch_size, 1))])

    #         labels_real = np.ones((batch_size, 1)) - 0.05*np.abs(np.random.random((batch_size, 1)))
    #         labels_fake = np.zeros((batch_size, 1)) + 0.05*np.abs(np.random.random((batch_size, 1)))

            labels_real = 0.9*np.ones((batch_size, 1)) 
            labels_fake = np.zeros((batch_size, 1)) 

            d_loss_real = D1.train_on_batch(real_images, labels_real)
            d_loss_fake = D1.train_on_batch(generated_images, labels_fake)
            d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

#             #### discriminator部分解鎖
#             for i in range(d_split_start,d_split_end):
#                 D1.layers[i].trainable=True

            random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

            misleading_targets = np.ones((batch_size, 1))

            g_loss = GAN1.train_on_batch(random_latent_vectors, misleading_targets)


            if low_step % 100 == 0:
                # Save model weights
                GAN1.save_weights('../../ta-hsi-datacenter/output_results/weight/gan1.h5')
                G1.save_weights('../../ta-hsi-datacenter/output_results/weight/g1.h5')
                D1.save_weights('../../ta-hsi-datacenter/output_results/weight/d1.h5')
                
                # Print metrics
                print('level_{}'.format(idx))
                print('low resolution, discriminator loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, d_loss))
                print('low resolution, adversarial loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, g_loss))
                display_grid = np.zeros((4*height_1,width_1,3))

                for j in range(4):
                    display_grid[j*height_1:(j+1)*height_1,0:width_1,:] = generated_images[j]
                
                img = image.array_to_img((display_grid[:,:,::-1]*127.5)+127.5, scale=False)
                img.save(os.path.join(save_dir, 'level_{}_low_generated_'.format(idx) + str(step*low_iteration+low_step+pre_low_step) + '.png'))
                print("--- %s seconds ---" % (time.time() - start_time))
                start_time = time.time()

    #         if num_split!=4:
    #             ## generator部分解鎖
    #             for k in range(g_split_start,g_split_end):
    #                 G1.layers[k].trainable=True
    #             ## discriminator部分解鎖
    #             for k in range(d_split_start,d_split_end):
    #                 D1.layers[k].trainable=True
            

Using TensorFlow backend.


level_0
low resolution, discriminator loss at step 0: 18.641347885131836
low resolution, adversarial loss at step 0: 1.3793268e-05
--- 10.761106967926025 seconds ---
level_0
low resolution, discriminator loss at step 100: 38.76502227783203
low resolution, adversarial loss at step 100: 7.9541604e-25
--- 23.474636793136597 seconds ---
level_0
low resolution, discriminator loss at step 200: 34.60512161254883
low resolution, adversarial loss at step 200: 5.2039624e-31
--- 23.48907470703125 seconds ---
level_0
low resolution, discriminator loss at step 300: 35.48690414428711
low resolution, adversarial loss at step 300: 1.2205809e-24
--- 31.613715887069702 seconds ---
level_0
low resolution, discriminator loss at step 400: 33.00086975097656
low resolution, adversarial loss at step 400: 3.2899313e-27
--- 28.74228310585022 seconds ---
level_0
low resolution, discriminator loss at step 500: 41.51607894897461
low resolution, adversarial loss at step 500: 1.5210889e-19
--- 23.717429876327515 sec

KeyboardInterrupt: 

In [ ]:
# G.get_layer(G.layers[5]._name)`

In [ ]:
# G.layers[5]._name